In [5]:
#imports 
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


In [13]:
teams_data

,id,first_name,height_feet,height_inches,last_name,position,team,weight_pounds,team_id
0,14,Ike,NaN,NaN,Anigbogu,C,IND,NaN,12
1,25,Ron,NaN,NaN,Baker,G,NYK,NaN,20
2,47,Jabari,NaN,NaN,Bird,G,BOS,NaN,2
3,67,MarShon,NaN,NaN,Brooks,G,MEM,NaN,15
4,71,Lorenzo,NaN,NaN,Brown,G,TOR,NaN,28
...,...,...,...,...,...,...,...,...,...
5245,19465585,Dru,NaN,NaN,Smith,G,MIA,NaN,16
5246,38017676,MarJon,NaN,NaN,Beauchamp,F,MIL,NaN,17
5247,38017677,Dyson,NaN,NaN,Daniels,G,NOP,NaN,19
5248,38017679,Jaden,NaN,NaN,Hardy,G,DAL,NaN,7


In [7]:
#data preprocessing 
games_data = pd.read_csv("games.csv")
teams_data = pd.read_csv("team.csv")
selected_columns = ['home_team_score', 'visitor_team_score', 'home_team_conference', 'visitor_team_conference']

selected_columns_games = ['home_team_score', 'visitor_team_score', 'home_team_id']

# Merge datasets
merged_data = pd.merge(games_data[selected_columns_games], teams_data, how='inner', left_on='home_team_id', right_on='team_id', suffixes=('_game', '_team'))
merged_data

,home_team_score,visitor_team_score,home_team_id,id,first_name,height_feet,height_inches,last_name,position,team,weight_pounds,team_id
0,126,94,2,47,Jabari,NaN,NaN,Bird,G,BOS,NaN,2
1,126,94,2,494,Michael,NaN,NaN,Smith,NaN,BOS,NaN,2
2,126,94,2,47,Jabari,NaN,NaN,Bird,G,BOS,NaN,2
3,126,94,2,494,Michael,NaN,NaN,Smith,NaN,BOS,NaN,2
4,126,94,2,669,John,NaN,NaN,Bagley,NaN,BOS,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...
12555699,81,101,50,46391409,Floyd,NaN,NaN,Volker,NaN,DN,NaN,50
12555700,81,101,50,46391417,Jack,NaN,NaN,Cotton,NaN,DN,NaN,50
12555701,81,101,50,46391430,Ed,NaN,NaN,Bartels,NaN,DN,NaN,50
12555702,81,101,50,46391434,Bill,NaN,NaN,Herman,NaN,DN,NaN,50


In [8]:
#1 for home win, 0 for away win
merged_data['outcome'] = (merged_data['home_team_score'] > merged_data['visitor_team_score']).astype(int)
merged_data = merged_data.drop(['home_team_score', 'visitor_team_score'], axis=1)

#split data
X = merged_data.drop("outcome", axis=1)
y = merged_data["outcome"]

# Identify categorical columns with high cardinality
high_cardinality_columns = [...]

# Label encode high cardinality columns
label_encoder = LabelEncoder()
for col in high_cardinality_columns:
    X[col] = label_encoder.fit_transform(X[col])

#train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

KeyError: Ellipsis